### Table of Contents

* [Relevant libraries and colour dictionaries](#chapter1)
* [Subsetting all slides: zoom](#chapter2)
* [SpatialDE](#chapter3)
* [Top30](#chapter4)
    * [Top 30 compare](#section_4_1)
* [Scatterplot](#chapter5)
* [AEH](#chapter6)
    * [AEH compare](#section_6_1)
* [Sophie's genes](#chapter7)
* [Zoom GA SA](#chapter8)
* [Granuloma borders](#chapter9)

#### Relevant libraries and colour dictionaries <a class="anchor" id="chapter1"></a>

In [ ]:
# Import relevant libraries
import numpy as np
import scanpy as sc
import os
import pandas as pd
import seaborn as sb
import matplotlib
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib import colors
import seaborn as sns
from collections import OrderedDict
from matplotlib import cm
import anndata as ann
import scanpy.external as sce
from datetime import datetime
import NaiveDE
import SpatialDE
from matplotlib_venn import venn3
%matplotlib inline

# Set current directory
os.chdir("/Users/mendenlab/work/spatial_granuloma/scripts")

# assign the rigth colours to the right annotation
def _set_colors(adata, obs_name, colors):
    """Set palette with specific colors for specific categories

    Parameters
    ----------
    adata : annData
    obs_name : column to plot
    colors : OrderedDict(): colors named by categories

    Returns
    -------

    """
    if len(colors.values())>0:
        palette = []
        unique_colors = np.unique(adata.obs[obs_name])
        for key in adata.obs[obs_name].cat.categories.tolist():
            if key in colors.keys():
                palette.append(colors[key])
    return palette

#Set the colours per annotation
spot_colors = []
spot_colors = OrderedDict()
spot_colors["EPIDERMIS"] = 'blue'
spot_colors["DERMIS"] = '#E0EEE0'
spot_colors["INTERFACE"] = 'deepskyblue'
spot_colors["VESSEL"] = 'darkgreen'
spot_colors["HAIR FOLLICLE"] = "#543005"
spot_colors["SWEAT GLAND"] = 'y'
spot_colors["SEBACEOUS GLAND"] = 'mistyrose'
spot_colors["MUSCLE"] = 'darkcyan'
spot_colors["GA"] = 'firebrick'  
spot_colors["GNL"] = 'orchid'
spot_colors["GSS"] = 'blueviolet'
spot_colors["GSC"] = 'mediumvioletred'
spot_colors["UNDETERMINED"] = 'black'


dermis_colors = []
dermis_colors = OrderedDict()
dermis_colors["UNDETERMINED"] = 'black'
dermis_colors["upper EPIDERMIS"] = 'blue'
dermis_colors["middle EPIDERMIS"] = 'dodgerblue'
dermis_colors["basal EPIDERMIS"] = 'skyblue'
dermis_colors["DERdepth1"] = '#006837'
dermis_colors["DERdepth2"] = '#238443'
dermis_colors["DERdepth3"] = '#41AB5D'
dermis_colors["DERdepth4"] = '#78C679'
dermis_colors["DERdepth5"] = '#ADDD8E'
dermis_colors["DERdepth6"] = '#D9F0A3'
dermis_colors["DERdepth7"] = '#F7FCB9'

leiden_r13_colours = []
leiden_r13_colours = OrderedDict()
leiden_r13_colours["0"] = 'darkolivegreen'
leiden_r13_colours["1"] = "#D9F0A3"
leiden_r13_colours["2"] = '#238443'
leiden_r13_colours["3"] = 'firebrick'
leiden_r13_colours["4"] = '#78C679'
leiden_r13_colours["5"] = '#78C679'
leiden_r13_colours["6"] = '#41AB5D'
leiden_r13_colours["7"] = '#006837'
leiden_r13_colours["8"] = '#ADDD8E'
leiden_r13_colours["9"] = "#238443"
leiden_r13_colours["10"] = '#78C679'
leiden_r13_colours["11"] = 'blue'
leiden_r13_colours["12"] = 'orchid'
leiden_r13_colours["13"] = '#F46D43'
leiden_r13_colours["14"] = 'dodgerblue'
leiden_r13_colours["15"] = 'deepskyblue'
leiden_r13_colours["16"] = '#cfafaf'
leiden_r13_colours["17"] = 'yellow'
leiden_r13_colours["18"] = 'darkcyan'
leiden_r13_colours["19"] = '#006837'


In [ ]:
def flatten(t):
    return [item for sublist in t for item in sublist]

In [ ]:
# Import adata 
adata_path = "../results/current/"

adata = sc.read(os.path.join(adata_path, "final/Granuloma_QC_clustering.h5"))
    
# setting up "factors" with different levels, order = TRUE
# add less common annotations LAST so they are not overwritten

# Set spot_type and skin_layer as categories and define the levels in each category
# Spot type: annatomical annotations
adata.obs['spot_type'] = pd.Categorical(
    adata.obs['spot_type'],
    categories = ['UNDETERMINED', 'DERMIS', "EPIDERMIS", 'INTERFACE', 'HAIR FOLLICLE',
                'VESSEL', 'MUSCLE', 'SEBACEOUS GLAND', 'SWEAT GLAND', 'GA', 'GNL', 'GSS', 'GSC'],
                 ordered = True)

# Skin layer
adata.obs['skin_layer'] = pd.Categorical(
    adata.obs['skin_layer'],
    categories = ['UNDETERMINED', 
                'upper EPIDERMIS', 'middle EPIDERMIS', 'basal EPIDERMIS',
                'DERdepth1', 'DERdepth2', 'DERdepth3', 'DERdepth4',
                'DERdepth5', 'DERdepth6', 'DERdepth7'],
    ordered = True)

In [ ]:
def index_spots(spot_coordinate, d):
    """Get index of nearest neighbor spots
        k: index of row
        m: index of column
        
    Parameters
    ----------
    spot_coordinate : numpy.array
        coordinate of spot of interest
    d : int
        distance of spot of interest to its next neighbor spots

    Returns
    -------

    """

    ind_list = []
    for k in range(-d, d + 1):
        m_max = 2 * d - np.abs(k)
        for m in range(-m_max, m_max + 1, 2):
            if m == 0 and k == 0:
                continue
            else:
                ind_list.append((spot_coordinate[0] + m, spot_coordinate[1] + k))

    return ind_list

In [ ]:
def get_location_spots(center_coord, distance, all_coordinates): 
    """Get index of spots in all_coordinates from center spot and nn spots

    Parameters
    ----------
    center_coord : tuple
        coordinate of center spot
    distance : int
        radius starting from center spot
    all_coordinates : list of tuples
        list containing all coordinates

    Returns
    -------

    """
    # 1. Get index of nn spots
    nn_indices = index_spots(center_coord, d=int(distance))

    # 2. Get index of nn spots in list of all coordinates and of cyto+ spots
    # 2.1 Index of cyto+ spots
    varindex_center_gene = np.where(np.all(np.array(all_coordinates) == np.array(center_coord), axis=1))[0]
    # 2.2 Indices of nn responder spots
    varindex_nn_genes = np.where(
        np.all(np.array(all_coordinates)[np.newaxis, :] == np.array(nn_indices)[:, np.newaxis], axis=2))[1]

    # Check if above command extracts nn spots positions in adata object -> yep, it does :)
    # np.array(all_coordinates)[varindex_responder_genes] == np.array(nn_indices)nn_indices

    return varindex_center_gene, varindex_nn_genes

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/SpatialDE/") # Set working directory so it saves it in the drive
print(os.getcwd())

#### Subsetting all slides: zoom <a class="anchor" id="chapter2"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_zoom_path)

# Parameters
# -------------------------------------------------------------------------------------------------------
# sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
number_neighbours = 2 # choose the number of neighbouring spots

# Loop
# -------------------------------------------------------------------------------------------------------
for sample_SPECIMEN in sample_SPECIMEN_list:
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    # Subset to only spots marked as GRANULOMA 
    adata_ga = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & (adata.obs['spot_type']== 'GA')].copy()  
    
    # Save coordinates of granuloma_spots+ spots of a sample in list of tuples [(x1, y1), (x2, y2), ...]
    print('Extracting coordinates of granuloma spots...')
    granuloma_spots_coordinates_df = pd.DataFrame()
    granuloma_spots_coordinates = []
    granuloma_spots_coordinates = list(zip(*(adata_ga.obs['array_col'].values, adata_ga.obs['array_row'].values)))
    granuloma_spots_coordinates_df = pd.DataFrame(granuloma_spots_coordinates, columns = ['array_col', 'array_row'])
    granuloma_spots_coordinates_df.to_csv(spatialde_zoom_path + date + '_granuloma_spots_coordinates_' + sample_SPECIMEN + '.csv')
    #granuloma_spots_coordinates
    
    # Get coordinates of neighbouring spots, take all unique values
    print('Extracting coordinates of neighbouring spots...')
    all_coordinates = []
    for ga_coord in granuloma_spots_coordinates:
        all_coordinates_temp = index_spots(ga_coord, number_neighbours)
        all_coordinates.append(all_coordinates_temp)

    all_coordinates = flatten(all_coordinates)
    print('Number of neighbouring spots: ' + str(len(set(all_coordinates))))
    all_coordinates_df = pd.DataFrame(all_coordinates, columns = ['array_col', 'array_row'])
    all_coordinates_df.to_csv(spatialde_zoom_path + date + '_all_coordinates_' + sample_SPECIMEN + '.csv')
    
    # Extract indexes
    all_indexes = []
    for coord in set(all_coordinates):
        #print(coord)
        index_temp = list(adata_sample_SPECIMEN.obs[(adata_sample_SPECIMEN.obs['array_col']== coord[0]) & (adata_sample_SPECIMEN.obs['array_row']== coord[1])].index)
        #print(index_temp)
        all_indexes.extend(index_temp)
    #all_indexes
    #len(all_indexes)
    textfile = open(spatialde_zoom_path + date + '_all_indexes_' + sample_SPECIMEN + '.txt', "w")
    for index in all_indexes:
        textfile.write(index + "\n")
    textfile.close()
    
    #adata2 = adata_sample_SPECIMEN.obs.loc[all_indexes] # This will only give us a dataframe of the observations, but we need the whole subset of the adata object, including vars! (genes)
#    print(list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_indexes))[0]))
    adata_zoom = adata_sample_SPECIMEN[list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_indexes))[0])]
    sc.write(os.path.join(spatialde_zoom_path, date + '_adata_zoom_' + sample_SPECIMEN + '.h5'), adata_zoom)
    
    # Plot and save figures
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, 80))
    
    plt.subplot(20, 3, 1)
    plt.scatter(adata_sample_SPECIMEN.obs['array_col'], 
                -1.5*adata_sample_SPECIMEN.obs['array_row'],
                c = adata_sample_SPECIMEN.obs['spot_type'].map(spot_colors))
    plt.title(sample_SPECIMEN)
    plt.colorbar(ticks=[]) 
    
    plt.subplot(20, 3, 2)
    plt.scatter(adata_ga.obs['array_col'], 
            -1.5*adata_ga.obs['array_row'],
            c = adata_ga.obs['spot_type'].map(spot_colors), s=80)
    plt.title(sample_SPECIMEN)
    plt.colorbar(ticks=[])
    
    plt.subplot(22, 3, 3)
    plt.scatter(adata_zoom.obs['array_col'], 
                -1.5*adata_zoom.obs['array_row'],
                c = adata_zoom.obs['spot_type'].map(spot_colors), s=80)
    plt.title(sample_SPECIMEN)
    plt.colorbar(ticks=[])
    plt.savefig(spatialde_zoom_path + date + '_Spatial_allandzoom_' + sample_SPECIMEN + '.png', bbox_inches='tight', dpi = 60)
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)

#### SpatialDE <a class="anchor" id="chapter3"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_zoom_path)

# Parameters
# -------------------------------------------------------------------------------------------------------
#sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
number_neighbours = 2 # choose the number of neighbouring spots

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    
    print('Starting analysis for: ' + sample_SPECIMEN)
    # Subset adata 
    #adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    # Subset to only spots marked as GRANULOMA 
    #adata_ga = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & (adata.obs['spot_type']== 'GA')].copy()  
    # Read in files  
    print('Careful with the date! Check that you are reading in the most recent file!')
    granuloma_spots_coordinates_df = pd.read_csv(spatialde_zoom_path + '20220504' + '_granuloma_spots_coordinates_' + sample_SPECIMEN + '.csv', index_col = 0)
    all_coordinates_df = pd.read_csv(spatialde_zoom_path + '20220504' + '_all_coordinates_' + sample_SPECIMEN + '.csv', index_col = 0)
    adata_zoom = sc.read(os.path.join(spatialde_zoom_path, '20220504' + '_adata_zoom_' + sample_SPECIMEN + '.h5'))
    
    # Get the counts dataframe
    counts = adata_zoom.to_df(layer="counts")
    counts = counts.T[counts.sum(0) >= 3].T  # Filter practically unobserved genes
    print(counts.shape)
    
    # Correct for library size or sequencing depth
    sample_info = sc.get.obs_df(adata_zoom, keys=['array_row', 'array_col', "n_counts"])
    counts = counts.loc[sample_info.index]  # Align count matrix with metadata table
    print(sample_info.head(5))
    norm_expr = NaiveDE.stabilize(counts.T).T
    resid_expr = NaiveDE.regress_out(sample_info, norm_expr.T, 'np.log(n_counts)').T
    
    # Save all the data
    sample_info.to_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + date + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts.to_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + date + '_counts_' + sample_SPECIMEN + '.csv')
    norm_expr.to_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + date + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr.to_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + date + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Run Spatial DEG
    print('Saving plot as: ' + spatialde_zoom_path + 'SpatialDE_Zoom/' + date + sample_SPECIMEN + '.csv')
    if not os.path.isfile(os.path.join(spatialde_zoom_path + 'SpatialDE_Zoom/' + '20220504' + sample_SPECIMEN + '.csv')):
        print('Calculating spatial DE for ' + sample_SPECIMEN)
        X = sample_info[['array_row', 'array_col']]
        results = SpatialDE.run(X, resid_expr)
        results.to_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + date + '_results_' + sample_SPECIMEN + '.csv')
        print('Wohoo!' + sample_SPECIMEN + ' data saved!')
        print("")
    
    # or load it if already computed
    else:
        print('Spatial DE already computed for ' + sample_SPECIMEN + ', importing results . csv file from: ' + spatialde_zoom_path)
        print('Need to load one by one, or write new code to load them all!:)')

#### Top30 <a class="anchor" id="chapter4"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_zoom_path)

# Parameters
# -------------------------------------------------------------------------------------------------------
#sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220504' # the timestamp of the files (check filename)

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    # Read in files: to be more computationally efficient, we will just read in the indexes and subset adata everytime.
    print('Careful with the date! Check that you are reading in the most recent file!')
    all_indexes = []
    with open(spatialde_zoom_path + timestamp + '_all_indexes_' + sample_SPECIMEN + '.txt') as f:
        all_indexes = f.read().splitlines()
    #print(all_indexes)
    adata_zoom = adata_sample_SPECIMEN[list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_indexes))[0])]
#     print(adata_zoom)
    
    # Load the data for the particular sample
    results = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Print results
    print(results.sort_values('qval').head(10)[['g', 'l', 'qval']])

    # Genes - top 30 by qval
    genes = results.sort_values('qval').head(30)['g'].tolist()
    textfile = open(spatialde_zoom_path + 'Top30/' + date + '_top30genes_' + sample_SPECIMEN + '.txt', "w")
    for gene in genes:
        textfile.write(gene + "\n")
    textfile.close()
    
    # Plot genes in spatial slides - top 30 by qval
    print('Plotting genes in spatial slides!')
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, 80))
    for i, g in enumerate(genes):
        plt.subplot(15, 2, 1)
        plt.scatter(adata_zoom.obs['array_col'], -1.5*adata_zoom.obs['array_row'], c = adata_zoom.obs['spot_type'].map(spot_colors))
        plt.axis('equal')
        plt.subplot(15, 2, i + 1)
        plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c=norm_expr[g], s = 80);
        plt.title(g)
        plt.axis('equal')
        plt.colorbar(ticks=[]);
    plt.savefig(spatialde_zoom_path + 'Top30/' + date + '_Top30qval_' + sample_SPECIMEN + '.pdf')
    plt.show()
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)

#### Top 30 compare  <a class="anchor" id="section_4_1"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names

# Parameters
# -------------------------------------------------------------------------------------------------------
#sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220504' # the timestamp of the files (check filename)

# Loop to read genes .txt file
# -------------------------------------------------------------------------------------------------------
all_top30genes = {}
for sample_SPECIMEN in sample_SPECIMEN_list:
    file_path = spatialde_zoom_path + 'Top30/' + timestamp + '_top30genes_' + sample_SPECIMEN + '.txt'
    
    with open(file_path) as f:
        all_top30genes[sample_SPECIMEN] = f.read().splitlines()
    
#print(all_top30genes)
df = pd.DataFrame(all_top30genes.items(), columns = ['sample_SPECIMEN', 'genes']).explode('genes')
df.genes.value_counts() > 1
v = df.genes.value_counts()
df_filtered = df[df.genes.isin(v.index[v.gt(1)])]
#df['genes'].value_counts().plot(kind='bar')

# Plot common genes
# -------------------------------------------------------------------------------------------------------
df_plot = df_filtered.groupby(['sample_SPECIMEN', 'genes']).size().reset_index().pivot(columns='sample_SPECIMEN', index='genes', values=0)
df_plot

plt.figure(facecolor='w', edgecolor='k', figsize=(60, 60))
df_plot.plot.bar(stacked = True, figsize=(8, 8),
                 color = ['dodgerblue', 'deepskyblue', 'lightblue', 'orange', 'gold', 'purple', 'fuchsia'])
plt.title("Common top 30 spatially-DE genes \n in zoomed-in granuloma spots (GA)")
plt.xlabel("Common genes")
plt.ylabel("Count")
plt.legend(loc='center left', bbox_to_anchor=(1.0, 0.75))
plt.savefig(spatialde_zoom_path + 'Top30/' + date + '_Barplot_common_top30' + '.png', bbox_inches='tight', dpi = 120)

# -----------------------------------------------------------------------------------------------------
# Another way of building the count plots
# sns.set()
# fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, 80))
# sns.countplot(y = df_filtered['genes'], 
#               color='darkblue', #hue = df_filtered['sample_SPECIMEN'], # for stacked or unstacked, by sample_SPECIMEN
#               orient = 'v',
#               order = df_filtered['genes'].value_counts().index)
# plt.show()


#### Scatterplots <a class="anchor" id="chapter5"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_zoom_path)

# Parameters
# -------------------------------------------------------------------------------------------------------
# sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220504' # the timestamp of the files (check filename)

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()

    # Read adata_zoom: to be more computationally efficient, we will just read in the indexes and subset adata everytime.
    print('Careful with the date! Check that you are reading in the most recent file!')
    all_indexes = []
    with open(spatialde_zoom_path + timestamp + '_all_indexes_' + sample_SPECIMEN + '.txt') as f:
        all_indexes = f.read().splitlines()
    adata_zoom = adata_sample_SPECIMEN[list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_indexes))[0])]
    
    # Load the data for the particular sample
    results = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
#     sample_info = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
#     counts = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
#     resid_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Highlight gene groups
    group1 = results.sort_values('FSV').tail(3)[['g', 'l', 'qval', 'FSV']] # highest FSV
    print(group1)
    group2 = results.sort_values('qval').head(3)[['g', 'l', 'qval', 'FSV']] # lowest q-value
    print(group2)    
    group3 = results[(results.qval > 1e-10) & (results.FSV < 0.98)].sort_values('FSV').tail(3)[['g', 'l', 'qval', 'FSV']]
    print(group3)
    group4 = results[(results.qval > 1e-15) & (results.FSV < 0.9)].sort_values(['qval', 'FSV'], ascending = [True, False]).head(3)[['g', 'l', 'qval', 'FSV']]
    print(group4)
    group5 = results[(results.qval < 1e-2) & (results.qval > 1e-4) & (results.FSV < 0.9)].sort_values(['FSV', 'qval'], ascending = [False, True]).head(3)[['g', 'l', 'qval', 'FSV']]
    print(group5)
    
    # Plot scatterplot
    print('Plotting scatterplot!')
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(10, 5))
    plt.yscale('log')
    plt.scatter(results['FSV'], 
                results['qval'] + 1e-20, 
                c = 'lightgray')
    plt.scatter(results.loc[results['g'].isin(group1.g)]['FSV'], 
                results.loc[results['g'].isin(group1.g)]['qval'] + 1e-20, # we have to sum 1e-18 to solve the problem of log(0)
                c = 'red')
    plt.scatter(results.loc[results['g'].isin(group2.g)]['FSV'], 
                results.loc[results['g'].isin(group2.g)]['qval'] + 1e-20, 
                c = 'purple')
    plt.scatter(results.loc[results['g'].isin(group3.g)]['FSV'], 
                results.loc[results['g'].isin(group3.g)]['qval'] + 1e-20, 
                c = 'green')
    plt.scatter(results.loc[results['g'].isin(group4.g)]['FSV'], 
                results.loc[results['g'].isin(group4.g)]['qval'] + 1e-20, 
                c = 'orange')
    plt.scatter(results.loc[results['g'].isin(group5.g)]['FSV'], 
                results.loc[results['g'].isin(group5.g)]['qval'] + 1e-20, 
                c = 'blue')

    plt.axhline(0.05, c='black', lw=1, ls='--')
    plt.axhline(0.01, c='black', lw=1, ls='--')
    
    plt.gca().invert_yaxis();
    plt.xlabel('Fraction spatial variance')
    plt.ylabel('Adj. P-value');
    plt.savefig(spatialde_zoom_path + 'Scatterplots/' + date + '_Scatterplot_' + sample_SPECIMEN + '.pdf')
    plt.show()
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)

#### AEH <a class="anchor" id="chapter6"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_zoom_path)

# Parameters
# -------------------------------------------------------------------------------------------------------
# sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220504' # the timestamp of the files we want to read in (check filename)
lengthscale_vector = [0.5]
conditions = ['qval < 0.01']
number_of_patterns = 4

# Loop for each sample_SPECIMEN
# -------------------------------------------------------------------------------------------------------
for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()

    # Read adata_zoom: to be more computationally efficient, we will just read in the indexes and subset adata everytime.
    print('Careful with the date! Check that you are reading in the most recent file!')
    all_indexes = []
    with open(spatialde_zoom_path + timestamp + '_all_indexes_' + sample_SPECIMEN + '.txt') as f:
        all_indexes = f.read().splitlines()
    adata_zoom = adata_sample_SPECIMEN[list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_indexes))[0])]
    
    # Load the data for the particular sample
    results = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    X = sample_info[['array_row', 'array_col']]
    
    # Loop for AEH
    for cond in conditions:
        print('Subsetting results for condition: ' + cond)
        print('')
        sign_results = results.query(cond)
        print('Saving df as: ' + spatialde_zoom_path + 'AEH/' + date + '_sign_results_' + cond + '.csv')
        sign_results.to_csv(spatialde_zoom_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_sign_results_' + cond + '.csv')

        for lengthscale in lengthscale_vector:
            print('calculating spatial de for lengthscale = ' + str(lengthscale))
            print('')
            histology_results, patterns = SpatialDE.aeh.spatial_patterns(X, resid_expr, sign_results, C=number_of_patterns, 
                                                                        l=lengthscale, verbosity=1)
            print('Saving spatial DE results')
            print('')
            patterns.to_csv(spatialde_zoom_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.csv')
            histology_results.to_csv(spatialde_zoom_path + 'AEH/' + date + '_' +  sample_SPECIMEN + '_histology_results_' + cond + '_lengthscale' + str(lengthscale) + '.csv')
            
            print('Computing plots')
            print('')
            fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, 12))
            for i in range(number_of_patterns):
                fig.suptitle(cond + '_lengthscale' + str(lengthscale), fontsize=16, y = 0.9)
                plt.subplot(int(number_of_patterns/2), 2, i + 1)
                plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c = patterns[i], s = 80); # Note: if patterns.csv is read in, you might need to change it to patterns[str(i)] 
                plt.axis('equal')
                plt.title('Pattern {} - {} genes'.format(i, histology_results.query('pattern == @i').shape[0] ))
                plt.colorbar(ticks=[]);
                plt.savefig(spatialde_zoom_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.png')
            print('Patterns successfully plotted and saved!')
            print('')
            
    print('Wohoo! AEH finished for: ' + sample_SPECIMEN + 'condition ' + cond)

#### Sophie's genes <a class="anchor" id="chapter7"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_zoom_path)

# Parameters and lists
# -------------------------------------------------------------------------------------------------------
#sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220504' # the timestamp of the files (check filename)
#Sophie's marker genes
marker_gene_list = ['TLR1', 'ITGAL', 'TNFRSF1B', 'CD86', 'SPI1', 'BTK', 
                    'IL10RA', 'CLDN7', 'NUDT11', 'MIGA1', 'TAP1', 'TAP2', 'GLI1'] #FAM73A not found, name for that gene is also MIGA1

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()
    # Read in files: to be more computationally efficient, we will just read in the indexes and subset adata everytime.
    print('Careful with the date! Check that you are reading in the most recent file!')
    all_indexes = []
    with open(spatialde_zoom_path + timestamp + '_all_indexes_' + sample_SPECIMEN + '.txt') as f:
        all_indexes = f.read().splitlines()
    #print(all_indexes)
    adata_zoom = adata_sample_SPECIMEN[list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_indexes))[0])]
#     print(adata_zoom)
    
    # Load the data for the particular sample
    results = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Some marker genes are not in norm_expr, take only those genes that are
    marker_gene_list_updated = list(set.intersection(set(marker_gene_list), set(norm_expr.columns.values.tolist())))
    no_of_rows = int((len(marker_gene_list_updated) + 2)/2)
    
    # Plot genes in spatial slides - top 30 by qval
    print('Plotting genes in spatial slides!')
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, 30))
    for i, g in enumerate(marker_gene_list_updated):
        plt.subplot(no_of_rows, 2, 1)
        plt.title(sample_SPECIMEN)
        plt.scatter(adata_zoom.obs['array_col'], -1.5*adata_zoom.obs['array_row'], c = adata_zoom.obs['spot_type'].map(spot_colors))
        plt.axis('equal')
        plt.subplot(no_of_rows, 2, 2)
        plt.scatter(adata_zoom.obs['array_col'], -1.5*adata_zoom.obs['array_row'], c = adata_zoom.obs['leiden_r1.3_patient'].map(leiden_r13_colours))
        plt.axis('equal')
        plt.subplot(no_of_rows, 2, i + 2)
        plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c=norm_expr[g], s = 80);
        plt.title(g)
        plt.axis('equal')
        plt.colorbar(ticks=[]);
    plt.savefig(spatialde_zoom_path + 'Sophies_marker_genes/' + date + '_Sophiesmarkergenes_'+ sample_SPECIMEN + '.png')
    plt.show()
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_zoom_path = "../output/SpatialDE/Zoom/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names

# Parameters and lists
# -------------------------------------------------------------------------------------------------------
# sample_SPECIMEN_list = ['P18554_1001_50107-A']
sample_SPECIMEN_list = ['P17851_1001_91253-A', 'P17851_1002_91253-A', 'P17851_1002_91253-B', 
                        'P17851_1003_45703-A', 'P17851_1004_45703-A', 'P18554_1001_50107-A',
                        'P18554_1002_50107-A']
timestamp = '20220504' # the timestamp of the files (check filename)

for sample_SPECIMEN in sample_SPECIMEN_list:
    norm_expr = pd.read_csv(spatialde_zoom_path + 'SpatialDE_Zoom/' + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    if 'NUDT11' in norm_expr.columns.values.tolist():
        print('Its here! ' + sample_SPECIMEN)
    else:
        print('not here for sample ' + sample_SPECIMEN)

In [ ]:
if 'NUDT11' in norm_expr.columns.values.tolist():
    print('Its here!')

In [ ]:
norm_expr.columns.values.tolist()

In [ ]:
list(set.intersection(set(marker_gene_list), set(norm_expr.columns.values.tolist())))

#### Zoom only center of GA and Sarcoidosis <a class="anchor" id="chapter8"></a>

## Zoom only center of GA and Sarcoidosis

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/Zoom_GA_SA/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P18554_1001_50107-A', 'P18554_1002_50107-A', 
                        'P18554_1006_82301-A', 'P18554_1006_82301-B']

# Loop
# -------------------------------------------------------------------------------------------------------
for sample_SPECIMEN in sample_SPECIMEN_list:
    
    print('Starting analysis for: ' + sample_SPECIMEN)
    # Subset to only spots from that sample_SPECIMEN marked as GRANULOMA 
    adata_subset = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & ((adata.obs['spot_type']== 'GA') | (adata.obs['spot_type']== 'GSC'))].copy() 

    # Get the counts dataframe
    counts = adata_subset.to_df(layer="counts")
    counts = counts.T[counts.sum(0) >= 3].T  # Filter practically unobserved genes
    print(counts.shape)
    
    # Correct for library size or sequencing depth
    sample_info = sc.get.obs_df(adata_subset, keys=['array_row', 'array_col', "n_counts"])
    counts = counts.loc[sample_info.index]  # Align count matrix with metadata table
    print(sample_info.head(5))
    norm_expr = NaiveDE.stabilize(counts.T).T
    resid_expr = NaiveDE.regress_out(sample_info, norm_expr.T, 'np.log(n_counts)').T
    
    # Save all the data
    sample_info.to_csv(spatialde_path + date + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts.to_csv(spatialde_path + date + '_counts_' + sample_SPECIMEN + '.csv')
    sc.write(os.path.join(spatialde_path, date + '_adata_subset_' + sample_SPECIMEN + '.h5'), adata_subset)
    norm_expr.to_csv(spatialde_path + date + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr.to_csv(spatialde_path + date + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Run Spatial DEG
    print('Saving plot as: ' + spatialde_path + date + sample_SPECIMEN + '.csv')
    if not os.path.isfile(os.path.join(spatialde_path + date + sample_SPECIMEN + '.csv')):
        print('Calculating spatial DE for patient 1 slide 1')
        X = sample_info[['array_row', 'array_col']]
        results = SpatialDE.run(X, resid_expr)
        results.to_csv(spatialde_path + date + '_results_' + sample_SPECIMEN + '.csv')
        print('Wohoo!' + sample_SPECIMEN + ' data saved!')
        print("")
    
    # or load it if already computed
    else:
        print('Spatial DE already computed for ' + sample_SPECIMEN + ', importing results . csv file from: ' + spatialde_path)
        print('Need to load one by one, or write new code to load them all!:)')


In [ ]:
# Visualise spatial slides
for sample_SPECIMEN in sample_SPECIMEN_list:
    adata_subset = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & ((adata.obs['spot_type']== 'GA') | (adata.obs['spot_type']== 'GSC'))].copy()
    fig = plt.figure(facecolor = 'w', edgecolor = 'k', figsize = (15, 7.5))
    plt.subplot(3, 3, 1)
    plt.title('coloured by spot type')
    plt.axis('equal')
    plt.scatter(adata_subset.obs['array_col'], -1.5*adata_subset.obs['array_row'], c = adata_subset.obs['spot_type'].map(spot_colors))
    plt.subplot(3, 3, 2)
    plt.title(sample_SPECIMEN + '\n coloured by skin layer')
    plt.axis('equal')
    plt.scatter(adata_subset.obs['array_col'], -1.5*adata_subset.obs['array_row'], c = adata_subset.obs['skin_layer'].map(dermis_colors))
    plt.subplot(3, 3, 3)
    plt.axis('equal')
    plt.title('coloured by leiden clusters (r = 1.3)')
    plt.scatter(adata_subset.obs['array_col'], -1.5*adata_subset.obs['array_row'], c = adata_subset.obs['leiden_r1.3_patient'].map(leiden_r13_colours))
    plt.axis('equal')
    plt.colorbar(ticks=[]);
    plt.savefig(spatialde_path + date + '_Slides_' + sample_SPECIMEN + '.png', bbox_inches='tight', dpi = 120)
    plt.show()

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/Zoom_GA_SA/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P18554_1001_50107-A', 'P18554_1002_50107-A', 
                        'P18554_1006_82301-A', 'P18554_1006_82301-B']

timestamp = '20220520' # the timestamp of the files (check filename)

# Loop
# -------------------------------------------------------------------------------------------------------

for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & ((adata.obs['spot_type']== 'GA') | (adata.obs['spot_type']== 'GSC'))].copy()
    
    # Load the data for the particular sample
    results = pd.read_csv(spatialde_path + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_path + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_path + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_path + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_path + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    # Print results
    print(results.sort_values('qval').head(10)[['g', 'l', 'qval']])

    # Genes - top 30 by qval
    genes = results.sort_values('qval').head(30)['g'].tolist()
    textfile = open(spatialde_path + 'Top30/' + date + '_top30genes_' + sample_SPECIMEN + '.txt', "w")
    for gene in genes:
        textfile.write(gene + "\n")
    textfile.close()
    
    # Plot genes in spatial slides - top 30 by qval
    print('Plotting genes in spatial slides!')
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(15, 60))
    for i, g in enumerate(genes):
        plt.subplot(15, 2, 1)
        plt.scatter(adata_sample_SPECIMEN.obs['array_col'], -1.5*adata_sample_SPECIMEN.obs['array_row'], c = adata_sample_SPECIMEN.obs['spot_type'].map(spot_colors))
        plt.axis('equal')
        plt.subplot(15, 2, i + 1)
        plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c=norm_expr[g], s = 80);
        plt.title(g)
        plt.axis('equal')
        plt.colorbar(ticks=[]);
    plt.savefig(spatialde_path + 'Top30/' + date + '_Top30qval_' + sample_SPECIMEN + '.png', bbox_inches='tight', dpi = 120)
    plt.show()
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_path = "../output/SpatialDE/Zoom_GA_SA/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_path)

# Subset by sample_SPECIMEN, then select the subsets we actually want to check - with GS or NL spots
sample_SPECIMEN_list = ['P18554_1001_50107-A', 'P18554_1002_50107-A', 
                        'P18554_1006_82301-A', 'P18554_1006_82301-B']

timestamp = '20220520' # the timestamp of the files (check filename)
lengthscale_vector = [1]
conditions = ['qval < 0.01']
number_of_patterns = 4

# Loop for each sample_SPECIMEN
# -------------------------------------------------------------------------------------------------------
for sample_SPECIMEN in sample_SPECIMEN_list:
    print('Starting analysis for: ' + sample_SPECIMEN) 
    # Subset adata 
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & ((adata.obs['spot_type']== 'GA') | (adata.obs['spot_type']== 'GSC'))].copy()

    # Load the data for the particular sample
    results = pd.read_csv(spatialde_path + timestamp + '_results_' + sample_SPECIMEN + '.csv')
    norm_expr = pd.read_csv(spatialde_path + timestamp + '_norm_expr_' + sample_SPECIMEN + '.csv')
    sample_info = pd.read_csv(spatialde_path + timestamp + '_sample_info_' + sample_SPECIMEN + '.csv')
    counts = pd.read_csv(spatialde_path + timestamp + '_counts_' + sample_SPECIMEN + '.csv')
    resid_expr = pd.read_csv(spatialde_path + timestamp + '_resid_expr_' + sample_SPECIMEN + '.csv')
    
    X = sample_info[['array_row', 'array_col']]
    
    # Loop for AEH
    for cond in conditions:
        print('Subsetting results for condition: ' + cond)
        print('')
        sign_results = results.query(cond)
        print('Saving df as: ' + spatialde_path + 'AEH/' + date + '_sign_results_' + cond + '.csv')
        sign_results.to_csv(spatialde_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_sign_results_' + cond + '.csv')

        for lengthscale in lengthscale_vector:
            print('calculating spatial de for lengthscale = ' + str(lengthscale))
            print('')
            histology_results, patterns = SpatialDE.aeh.spatial_patterns(X, resid_expr, sign_results, C=number_of_patterns, 
                                                                        l=lengthscale, verbosity=1)
            print('Saving spatial DE results')
            print('')
            patterns.to_csv(spatialde_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.csv')
            histology_results.to_csv(spatialde_path + 'AEH/' + date + '_' +  sample_SPECIMEN + '_histology_results_' + cond + '_lengthscale' + str(lengthscale) + '.csv')
            
            print('Computing plots')
            print('')
            fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, int(3*number_of_patterns)))
            for i in range(number_of_patterns):
                fig.suptitle(cond + '_lengthscale' + str(lengthscale), fontsize=16, y = 0.9)
                plt.subplot(int(number_of_patterns/2), 2, i + 1)
                plt.scatter(sample_info['array_col'], -1.5*sample_info['array_row'], c = patterns[i], s = 80); # Note: if patterns.csv is read in, you might need to change it to patterns[str(i)] 
                plt.axis('equal')
                plt.title('Pattern {} - {} genes'.format(i, histology_results.query('pattern == @i').shape[0] ))
                plt.colorbar(ticks=[]);
                plt.savefig(spatialde_path + 'AEH/' + date + '_' + sample_SPECIMEN + '_patterns_' + cond + '_lengthscale' + str(lengthscale) + '.png', bbox_inches='tight', dpi = 120)
            print('Patterns successfully plotted and saved!')
            print('')
            
    print('Wohoo! AEH finished for: ' + sample_SPECIMEN + 'condition ' + cond)

In [ ]:
adata

#### Adding granuloma border <a class="anchor" id="chapter9"></a>

In [ ]:
os.chdir("/Volumes/Drive/spatial_granuloma/output/") # Set working directory so it saves it in the drive
spatialde_border_path = "../output/SpatialDE/Border_granulomas/"
date = datetime.now().strftime("%Y%m%d") #For timestamp in file names
print('Saving plots in: ' + spatialde_border_path)

# Parameters
# -------------------------------------------------------------------------------------------------------
number_of_neighbours = 2 # Number of neighbours for the first layer of the border

# We will only run this for sample_SPECIMENs that have lesional spots, and exclude the 0 specimens
sample_SPECIMEN_list = list(adata[(adata.obs['LESIONAL'] == 1) & (adata.obs['SPECIMEN'].str.contains('-0')==False)].obs['sample_SPECIMEN'].unique())
sample_SPECIMEN_list.remove('P17851_1001_91253-B') # this one does not have any granuloma spots 

# Loop
# -------------------------------------------------------------------------------------------------------

#for debug
#sample_SPECIMEN_list = ['P18554_1001_50107-A']

all_border_indexes = []
for sample_SPECIMEN in sample_SPECIMEN_list:
    # Subset to that particular sample_SPECIMEN
    adata_sub = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN)].copy()

    # Subset to only spots marked as GRANULOMA from that particular sample_SPECIMEN
    adata_sample_SPECIMEN = adata[(adata.obs['sample_SPECIMEN']== sample_SPECIMEN) & ((adata.obs['spot_type']== 'GA') | 
                                                                                      (adata.obs['spot_type']== 'GSC') | 
                                                                                      (adata.obs['spot_type']== 'GSS') | 
                                                                                      (adata.obs['spot_type']== 'GNL'))].copy()
    # Save coordinates of granuloma_spots+ spots of a sample in list of tuples [(x1, y1), (x2, y2), ...]
    print('Extracting coordinates of granuloma spots...' + sample_SPECIMEN)
    granuloma_spots_coordinates_df = pd.DataFrame()
    granuloma_spots_coordinates = []
    # granuloma_spots_coordinates is a list of the coordinates in format (x,y)
    granuloma_spots_coordinates = list(zip(*(adata_sample_SPECIMEN.obs['array_col'].values, adata_sample_SPECIMEN.obs['array_row'].values)))
    # granuloma_spots_coordinates_df is a dataframe of the coordinates with one column for x coord and one column for y coord
    granuloma_spots_coordinates_df = pd.DataFrame(granuloma_spots_coordinates, columns = ['array_col', 'array_row'])
    print('size of granuloma_spots_coordinates_df: ' + str(granuloma_spots_coordinates_df.size))
        
    # Get coordinates of neighbouring spots, take all unique values
    print('Extracting coordinates of neighbouring spots...')
    all_coordinates = []
    for g_coord in granuloma_spots_coordinates:
        all_coordinates_temp = index_spots(g_coord, 1)
        all_coordinates.append(all_coordinates_temp)
    
    all_coordinates = flatten(all_coordinates)
    print('Number of neighbouring spots: ' + str(len(set(all_coordinates))))
    # all_coordinates_df contains the granuloma spots and the neighbouring ones (just 1)
    all_coordinates_df = pd.DataFrame(all_coordinates, columns = ['array_col', 'array_row'])
    all_coordinates_df.to_csv(spatialde_border_path + date + '_all_coordinates_' + sample_SPECIMEN + '.csv')
    #print(all_coordinates_df)
    
    # Now take only the first outer layer of the granuloma spot (substract actual granuloma spots)
    df_all = all_coordinates_df.merge(granuloma_spots_coordinates_df.drop_duplicates(), on=['array_col', 'array_row'], 
                   how='left', indicator=True)
    only_first_layer_df = df_all[df_all['_merge'] == 'left_only'].drop_duplicates()
    only_first_layer_coordinates = list(set(all_coordinates) - set(granuloma_spots_coordinates))
    print('Number of spots in the first layer of the border: ' + str(len(only_first_layer_coordinates)))
    
    # Take again the next 2 layers of the first border
    print('Extracting coordinates of neighbouring spots of the border...')
    all_b_coordinates = []
    for b_coord in only_first_layer_coordinates:
        all_b_coordinates_temp = index_spots(b_coord, number_of_neighbours)
        all_b_coordinates.append(all_b_coordinates_temp)
    all_b_coordinates = flatten(all_b_coordinates)
    print('Number of neighbouring spots of the border: ' + str(len(set(all_b_coordinates))))
    
    # Extract indexes to be able to subset adata easier
    all_b_indexes = [] # contains all the border indexes of that sample_SPECIMEN
    for coord in set(all_b_coordinates):
        #print(coord)
        index_temp = list(adata_sample_SPECIMEN.obs[(adata_sample_SPECIMEN.obs['array_col']== coord[0]) & (adata_sample_SPECIMEN.obs['array_row']== coord[1])].index)
        #print(index_temp)
        all_b_indexes.extend(index_temp)
    textfile = open(spatialde_border_path + date + '_all_b_indexes_' + sample_SPECIMEN + '.txt', "w")
    for index in all_b_indexes:
        textfile.write(index + "\n")
    textfile.close()
    all_border_indexes.append(all_b_indexes) # will contain all border indexes of all sample_SPECIMENs
    
    # subsetting adata to get the anndata of only those border spots 
    adata_zoom = adata_sample_SPECIMEN[list(np.where(adata_sample_SPECIMEN.obs.index.isin(all_b_indexes))[0])]
    
    # Plot and save figures
    fig = plt.figure(facecolor='w', edgecolor='k', figsize=(20, 80))
    
    plt.subplot(20, 3, 1)
    plt.scatter(adata_sub.obs['array_col'], 
                -1.5*adata_sub.obs['array_row'],
                c = adata_sub.obs['spot_type'].map(spot_colors))
    plt.title(sample_SPECIMEN)
    plt.colorbar(ticks=[]) 
    
    plt.subplot(22, 3, 2)
    plt.scatter(adata_zoom.obs['array_col'], 
                -1.5*adata_zoom.obs['array_row'],
                c = adata_zoom.obs['spot_type'].map(spot_colors), s=80)
    plt.title(sample_SPECIMEN)
    plt.colorbar(ticks=[])
    
    plt.subplot(22, 3, 3)
    plt.scatter(adata_zoom.obs['array_col'], 
                -1.5*adata_zoom.obs['array_row'],
                c = adata_zoom.obs['leiden_r1.3_patient'].map(leiden_r13_colours), s=80)
    plt.title(sample_SPECIMEN)
    plt.colorbar(ticks=[])
    
    plt.savefig(spatialde_border_path + date + '_Spatial_border_' + sample_SPECIMEN + '.png', bbox_inches='tight', dpi = 60)
    
    print('Wohoo! Plots saved for: ' + sample_SPECIMEN)


all_border_indexes = flatten(all_border_indexes)

In [ ]:
textfile = open(spatialde_border_path + date + '_all_border_indexes_' + sample_SPECIMEN + '.txt', "w")

for index in all_border_indexes:
    textfile.write(index + "\n")
textfile.close()

## Adding border spots as an annotation in adata.obs for future use

In [ ]:
# create a new column in adata.obs called 'Border_spot' which has value 1 if it is a border spot (indexes of those spots are saved in all_border_indexes) and 
adata.obs['Border_spot'] = np.where(adata.obs.index.isin(all_border_indexes), 1, 0) 
adata.obs.head(5)

# !!!!! Attention !!!! Careful with overwritting !!!!!
# Make sure you have saved the previous adata object in the folder before running the following lines!!!
os.chdir("/Users/mendenlab/work/spatial_granuloma/scripts")
adata_path = "../results/current/"
#adata = sc.write(os.path.join(adata_path, "final/Granuloma_QC_clustering.h5"), adata)